# Load Transactions

In [6]:
from pathlib import Path
from datetime import datetime

import pandas as pd


def str2float(x):
    if not isinstance(x, str):
        return float(x)
    return float(x.replace(',', ''))

path = Path('/Users/ab011sw/Documents/Personal/investments/coinmate.tsv')
transactions = pd.read_csv(path, sep='\t')
transactions['Date'] = transactions['Date']\
    .apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))\

transactions['Price'] = transactions['Price'].apply(str2float)
transactions['Fee'] = transactions['Fee'].apply(str2float)

In [7]:
transactions.head()

,Date,Code,Currency,Price,Quantity,Action,Fee
0,2021-04-16,BTC,CZK,1343533.00,0.000104,buy,0.488247
1,2020-08-17,BTC,CZK,270156.00,0.044789,sell,14.519982
2,2019-08-12,BTC,CZK,262801.23,0.044789,buy,29.426432
3,2019-08-07,BTC,CZK,275123.00,0.042808,sell,14.132929
4,2019-08-07,BTC,CZK,267971.99,0.042808,buy,28.678302


# Process Data

In [8]:
from convertor.yahoo_convertor import YahooConvertor

convertors = {
    'CZK': YahooConvertor(ticker='CZK=X', inverse=True),
    'BTC': YahooConvertor(ticker='BTC-USD'),
}

result = pd.DataFrame([], columns=transactions.columns)

for _, row in transactions.iterrows():
    convertor = convertors[row['Currency']]
    new_row = row.copy()
    new_row['Price'] = convertor.convert(row['Price'], row['Date'])
    new_row['Fee'] = convertor.convert(row['Fee'], row['Date'])
    new_row['Currency'] = 'USD'
    result = result.append(new_row)

result

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Code,Currency,Price,Quantity,Action,Fee
0,2021-04-16,BTC,USD,62068.419107,0.000104,buy,0.022556
1,2020-08-17,BTC,USD,12265.879682,0.044789,sell,0.659250
2,2019-08-12,BTC,USD,11420.678371,0.044789,buy,1.278798
3,2019-08-07,BTC,USD,11981.143579,0.042808,sell,0.615465
4,2019-08-07,BTC,USD,11669.729129,0.042808,buy,1.248892
5,2019-08-05,BTC,USD,11446.958249,0.033182,sell,0.455796
6,2019-08-05,BTC,USD,11446.958249,0.001645,sell,0.022591
7,2019-08-05,BTC,USD,11446.958249,0.008687,sell,0.119332
8,2019-07-13,BTC,USD,11395.921869,0.043514,buy,1.239697
9,2019-07-12,BTC,USD,11751.484756,0.042462,sell,0.598790


In [9]:
row

Date        2017-09-11 00:00:00
Code                        BTC
Currency                    CZK
Price                  96713.77
Quantity               0.206075
Action                      buy
Fee                   69.755854
Name: 31, dtype: object

In [10]:
new_row

Date        2017-09-11 00:00:00
Code                        BTC
Currency                    USD
Price               4453.161893
Quantity               0.206075
Action                      buy
Fee                    3.211891
Name: 31, dtype: object